# Best window optimization

## Table of content

- [Imports](#imports)
- [Loading and pre-processing data](#loading)
- [Get functions](#functions)
- [Best window Optimization Algorithm](#algo)

Note: The main section is the last. [Go to the next section](#algo) to understand what is going on in this notebook.

## <a id="imports"></a>Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import helper

## <a id="loading"></a>Data loading

In [1]:
%time
START_DATE='1969-01-01'
END_DATE='2023-05-18'

# Load data
raw_excess_returns = pd.read_excel('data.xlsx', sheet_name='Return Indices')

# Set 'Dates' column as index
raw_excess_returns.set_index('Dates', inplace=True)

# Replace consecutive spaces with underscores in column names
raw_excess_returns.columns = raw_excess_returns.columns.str.replace(r'\s+', '_').str.upper()

# Convert the index to datetime
raw_excess_returns.index = pd.to_datetime(raw_excess_returns.index)

raw_excess_returns = raw_excess_returns.loc[START_DATE:END_DATE]

# divide by 100 returns
raw_excess_returns = raw_excess_returns.div(100)-1

# replace NaN with 0
raw_excess_returns = raw_excess_returns.fillna(0)  

# get a list of all the assets
assets_list = raw_excess_returns.columns.values

# calculate the commodity market factor
commodity_market_factor = raw_excess_returns.mean(axis=1).to_frame()
commodity_market_factor.index.name = 'Date'
commodity_market_factor.rename(columns = {list(commodity_market_factor)[0]: 'commodity_market_factor'}, inplace = True)

# # Add the 'commodity_market_factor' column to the DataFrame
# raw_excess_returns['commodity_market_factor'] = commodity_market_factor

# convert returns to log returns
excess_returns = np.log(1+raw_excess_returns)

log_commodity_market_factor = np.log(1+commodity_market_factor)


CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.05 µs


<ipython-input-1-fae6f177d4f8>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  raw_excess_returns.columns = raw_excess_returns.columns.str.replace(r'\s+', '_').str.upper()
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


## <a id="functions"></a>Get functions

In [2]:
%time
def high_low_breakout(excess_returns, window=40):
    """
    Parameters:
        excess_returns (pandas.DataFrame)
        window (int)
        
    Returns:
        signals (pandas.Dataframe)
    """

    signals = pd.DataFrame(index=excess_returns.index, columns=excess_returns.columns)

    for timestamp in excess_returns.index:

        for ticker in excess_returns.columns:
            signal = 0 # No signal (yet)
            current_excess_return = excess_returns.loc[timestamp, ticker]
            highest_return = excess_returns[ticker].rolling(window).max().loc[timestamp]
            lowest_return = excess_returns[ticker].rolling(window).min().loc[timestamp]

            if current_excess_return == highest_return:
                signal = 1 # Buy signal

            if current_excess_return == lowest_return:
                signal = -1 # Sell signal

            signals.loc[timestamp, ticker] = signal
    return signals

def convert_signal_to_position(signals):
    """
    Parameters:
        signals (pandas.DataFrame)
        
    Returns:
        converted_signals (pandas.Dataframe)
    """
    converted_signals = pd.DataFrame(index=signals.index, columns=signals.columns)
    for col in signals.columns:
        new_series = pd.Series(index=signals.index, dtype=int)
        position = ''
        state = 'hold'
        for idx,i in signals[col].items():
            if i == 0:
                state = 'hold'
                new_series[idx] = state
            elif i == 1:
                if position == 'buy':
                    state = 'hold'
                    new_series[idx] = state
                else:
                    position = 'buy'
                    state = 'buy'
                    new_series[idx] = state
            elif i == -1:
                if position == 'sell':
                    state = 'hold'
                    new_series[idx] = state
                else:
                    position = 'sell'
                    state = 'sell'
                    new_series[idx] = state
        converted_signals[col] = new_series
    return converted_signals

def calculate_max_drawdown(returns):
    """
    Parameters:
        returns (pandas.Series)
        
    Returns:
        (float). refers to max drawdown
        drawdown_start_date (datetime64[ns]). format: YYYY-MM-DD.
        drawdown_end_date (datetime64[ns]). format: YYYY-MM-DD.
    """
    #check if the returns is a pandas.core.series.Series
    if not isinstance(returns, pd.Series):
        raise TypeError("returns must be a pandas.core.series.Series")
    drawdowns = []
    drawdown = returns[0]
    maxi = returns[0]
    drawdown_end_date = returns.index[0]
    drawdown_start_date = returns.index[0]

    for date,ret in returns.items():
        if ret > maxi: 
            maxi = ret
            drawdown_start_date = returns.index[0]

            
        if ret < drawdown: 
            drawdown = ret 
            drawdown_end_date = date
            
        drawdowns.append(drawdown)
        
    return -np.linalg.norm(maxi-drawdown), drawdown_start_date, drawdown_end_date
    
def calculate_kurtosis(returns):
    """
    Parameters:
        returns (pandas.Series)
        
    Returns:
        kurtosis (float)
    """
    
    #check if the returns is a pandas.core.series.Series
    if not isinstance(returns, pd.Series):
        raise TypeError("returns must be a pandas.core.series.Series")

    #calculate
    kurtosis = returns.kurtosis()

    #output
    return kurtosis

def calculate_skewness(returns):
    """
    Parameters:
        returns (pandas.Series)
        
    Returns:
        skewness (float)
    """
    
    #check if the returns is a pandas.core.series.Series
    if not isinstance(returns, pd.Series):
        raise TypeError("returns must be a pandas.core.series.Series")

    #calculate
    skewness = returns.skew()

    #output
    return skewness

def calculate_win_rate(returns):
    """
    Parameters:
        returns (pandas.Series)
        
    Returns:
        win_rate (float)
    """
    #check if the returns is a pandas.core.series.Series
    if not isinstance(returns, pd.Series):
        raise TypeError("returns must be a pandas.core.series.Series")

    #calculate
    win_rate = (returns > 0).sum() / len(returns)
    # Return the win rate
    return win_rate


def compute_rolling_table(converted_signals):
    """
    Parameters:
        converted_signals (pandas.DataFrame)
        
    Returns:
        rolling_table (pandas.DataFrame)
    """
    
    cumulative_returns = pd.Series(0, index=converted_signals.index)
    rolling_table = []
    pending_tx = {}

    for ticker in converted_signals.columns:
        opened_or_closed = 'closed'
        # filter out hold to only keep buy/sell:
        for date, buy_or_sell in converted_signals[ticker][converted_signals[ticker] != 'hold'].items():
            if opened_or_closed == 'closed':
                opened_or_closed = 'opened'
                opened_date = date
                pending_tx[ticker] = opened_date
                rolling_table.append({
                    'Date': opened_date,
                    'Ticker': ticker,
                    'buy_or_sell': buy_or_sell,
                    'opened_or_closed': opened_or_closed,
                    'return': 0,
                    'max_drawdown': 0,
                    'max_drawdown_start_date': None,
                    'max_drawdown_end_date': None,
                    'kurtosis': 0,
                    'skewness': 0,
                    'win_rate': 0,
                })
            elif opened_or_closed == 'opened':

                opened_or_closed = 'closed'
                opened_date = pending_tx[ticker]
                closed_date = date
                rolling_table.append({
                    'Date': closed_date,
                    'Ticker': ticker,
                    'buy_or_sell': buy_or_sell,
                    'opened_or_closed': opened_or_closed,
                    'return': excess_returns[ticker].loc[opened_date:closed_date].sum(),
                    'max_drawdown': calculate_max_drawdown(excess_returns[ticker].loc[opened_date:closed_date])[0],
                    'max_drawdown_start_date': calculate_max_drawdown(excess_returns[ticker].loc[opened_date:closed_date])[1],
                    'max_drawdown_end_date': calculate_max_drawdown(excess_returns[ticker].loc[opened_date:closed_date])[2],
                    'kurtosis': calculate_kurtosis(excess_returns[ticker].loc[opened_date:closed_date]),
                    'skewness': calculate_skewness(excess_returns[ticker].loc[opened_date:closed_date]),
                    'win_rate': calculate_win_rate(excess_returns[ticker].loc[opened_date:closed_date]),
                })
                del pending_tx[ticker]


    rolling_table = pd.DataFrame(rolling_table)
    rolling_table = rolling_table.sort_values(by='Date', ascending=True).reset_index(drop=True)
    rolling_table['cumulative return'] = rolling_table['return'].cumsum()
    return rolling_table

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs


## <a id="algo"></a>Best window Optimization Algorithm
The below  is used to export excel files that will be used for further analysis and selection of best window size in our high_low_breakout strategy.
The process of selection of best window size is the following:
1. create a list of windows (from 5 to 200 with increment 5)
2. for each window, 
    2.a run the strategy 
    2.b compute the strategy rolling table
    2.c compute the strategy performance
    2.d merge the performance with the benchmark in one df
3. export `rolling_table`, `performance` and `merged_df`

In [3]:
iteration_window = [i for i in range(5, 201) if i % 5 == 0]
for iter_window in iteration_window:
    print(f'//========= START OF ITERATION : {iter_window} ========= \\')
    %time
    signals = high_low_breakout(excess_returns, iter_window)
    converted_signals = convert_signal_to_position(signals)
    rolling_table = compute_rolling_table(converted_signals)
    performance = pd.DataFrame(rolling_table.groupby('Date')['non_weighted_return'].sum())
    performance.rename(columns = {list(performance)[0]: 'strategy_return'}, inplace = True)
#     helper.plot_df(pd.merge(commodity_market_factor.cumsum(), performance.cumsum(), on='Date'), title='performance')
    try:
        rolling_table.to_excel(f'rolling_table{iter_window}.xlsx')
        print(f"Excel file created successfully: rolling_table{iter_window}.xlsx")
    except:
        print(f"ERROR: Could not create excel file: rolling_table{iter_window}.xlsx")
    try:
        performance.to_excel(f'performance{iter_window}.xlsx')
        print(f"Excel file created successfully: performance{iter_window}.xlsx")
    except:
        print(f"ERROR: Could not create excel file: performance{iter_window}.xlsx")
    merged_df = pd.merge(commodity_market_factor.cumsum(), performance.cumsum(), on='Date')
    try:
        merged_df.to_excel(f'merged_df{iter_window}.xlsx')
        print(f"Excel file created successfully: merged_df{iter_window}.xlsx")
    except:
        print(f"ERROR: Could not create excel file: merged_df{iter_window}.xlsx")
    print(f'//========= END OF ITERATION : {iter_window} ========= \\')


//========= START OF ITERATION : 5 ========= \
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


<ipython-input-3-59eb44f7b863>:12: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  rolling_table.to_excel(f'rolling_table{iter_window}.xlsx')


Excel file created successfully: rolling_table5.xlsx


<ipython-input-3-59eb44f7b863>:17: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  performance.to_excel(f'performance{iter_window}.xlsx')


Excel file created successfully: performance5.xlsx


...

Excel file created successfully: merged_df195.xlsx
//========= END OF ITERATION : 195 ========= \
//========= START OF ITERATION : 200 ========= \
CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 3.81 µs


<ipython-input-3-59eb44f7b863>:12: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  rolling_table.to_excel(f'rolling_table{iter_window}.xlsx')
<ipython-input-3-59eb44f7b863>:17: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  performance.to_excel(f'performance{iter_window}.xlsx')


Excel file created successfully: rolling_table200.xlsx
Excel file created successfully: performance200.xlsx


<ipython-input-3-59eb44f7b863>:23: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  rolling_table.to_excel(f'merged_df{iter_window}.xlsx')


Excel file created successfully: merged_df200.xlsx
//========= END OF ITERATION : 200 ========= \
